# Labelme转YOLO-批量

同济子豪兄 2023-4-16

## 导入工具包

In [1]:
import os
import json
import shutil
import numpy as np
from tqdm import tqdm

## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [2]:
!find . -iname '__MACOSX'

FIND: 参数格式不正确


In [3]:
!find . -iname '.DS_Store'

FIND: 参数格式不正确


In [4]:
!find . -iname '.ipynb_checkpoints'

FIND: 参数格式不正确


### 删除多余文件

In [5]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

此时不应有 i。


In [6]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

此时不应有 i。


In [7]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

此时不应有 i。


### 验证多余文件已删除

In [8]:
!find . -iname '__MACOSX'

FIND: 参数格式不正确


In [9]:
!find . -iname '.DS_Store'

FIND: 参数格式不正确


In [10]:
!find . -iname '.ipynb_checkpoints'

FIND: 参数格式不正确


## 指定数据集信息

In [11]:
Dataset_root = 'target_dataset'

In [12]:
classes = {
    't_blue':0,
    't_green':1,
    't_red':2,
}

## 生成`classes.txt`文件

In [13]:
os.chdir(Dataset_root)

In [14]:
with open('classes.txt', 'w', encoding='utf-8') as f:
    for each in list(classes.keys()):
        f.write(each + '\n')

In [15]:
os.mkdir('labels')
os.mkdir('labels/train')
os.mkdir('labels/val')

## 函数-处理单个labelme标注json文件

In [16]:
def process_single_json(labelme_path, save_folder='../../labels/train'):
    
    # 载入 labelme格式的 json 标注文件
    with open(labelme_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
        
    img_width = labelme['imageWidth']   # 图像宽度
    img_height = labelme['imageHeight'] # 图像高度
    
    # 生成 YOLO 格式的 txt 文件
    suffix = labelme_path.split('.')[-2]
    yolo_txt_path = suffix + '.txt'
    
    with open(yolo_txt_path, 'w', encoding='utf-8') as f:
        for each_ann in labelme['shapes']: # 遍历每个框

            if each_ann['shape_type'] == 'rectangle': # 筛选出框

                # 获取类别 ID
                bbox_class_id = classes[each_ann['label']]

                # 左上角和右下角的 XY 像素坐标
                bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
                bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))

                # 框中心点的 XY 像素坐标
                bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
                bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)

                # 框宽度
                bbox_width = bbox_bottom_right_x - bbox_top_left_x

                # 框高度
                bbox_height = bbox_bottom_right_y - bbox_top_left_y

                # 框中心点归一化坐标
                bbox_center_x_norm = bbox_center_x / img_width
                bbox_center_y_norm = bbox_center_y / img_height

                # 框归一化宽度
                bbox_width_norm = bbox_width / img_width
                # 框归一化高度
                bbox_height_norm = bbox_height / img_height

                # 生成 YOLO 格式的一行标注，指定保留小数点后几位
                bbox_yolo_str = '{} {:.4f} {:.4f} {:.4f} {:.4f}'.format(bbox_class_id, bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)
                # 写入 txt 文件中
                f.write(bbox_yolo_str + '\n')

    shutil.move(yolo_txt_path, save_folder)
    print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))

## 转换训练集标注文件至`labels/train`目录

In [17]:
os.chdir('labelme_jsons/train')

In [18]:
save_folder = '../../labels/train'
for labelme_path in os.listdir():
    process_single_json(labelme_path, save_folder=save_folder)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

0.json --> 0.txt 转换完成
1.json --> 1.txt 转换完成
100.json --> 100.txt 转换完成
101.json --> 101.txt 转换完成
102.json --> 102.txt 转换完成
103.json --> 103.txt 转换完成
104.json --> 104.txt 转换完成
105.json --> 105.txt 转换完成
107.json --> 107.txt 转换完成
108.json --> 108.txt 转换完成
109.json --> 109.txt 转换完成
11.json --> 11.txt 转换完成
110.json --> 110.txt 转换完成
111.json --> 111.txt 转换完成
112.json --> 112.txt 转换完成
113.json --> 113.txt 转换完成
114.json --> 114.txt 转换完成
115.json --> 115.txt 转换完成
116.json --> 116.txt 转换完成
117.json --> 117.txt 转换完成
118.json --> 118.txt 转换完成
119.json --> 119.txt 转换完成
121.json --> 121.txt 转换完成
122.json --> 122.txt 转换完成
123.json --> 123.txt 转换完成
124.json --> 124.txt 转换完成
125.json --> 125.txt 转换完成
126.json --> 126.txt 转换完成
127.json --> 127.txt 转换完成
128.json --> 128.txt 转换完成
129.json --> 129.txt 转换完成
13.json --> 13.txt 转换完成
130.json --> 130.txt 转换完成
131.json --> 131.txt 转换完成
134.json --> 134.txt 转换完成
135.json --> 135.txt 转换完成
136.json --> 136.txt 转换完成
138.json --> 138.txt 转换完成
139.json --> 139.txt 转换完

In [19]:
os.chdir('../../')

## 转换测试集标注文件至`labels/val`目录

In [20]:
os.chdir('labelme_jsons/val')

In [21]:
save_folder = '../../labels/val'
for labelme_path in os.listdir():
    process_single_json(labelme_path, save_folder=save_folder)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

10.json --> 10.txt 转换完成
106.json --> 106.txt 转换完成
12.json --> 12.txt 转换完成
120.json --> 120.txt 转换完成
132.json --> 132.txt 转换完成
133.json --> 133.txt 转换完成
137.json --> 137.txt 转换完成
141.json --> 141.txt 转换完成
149.json --> 149.txt 转换完成
150.json --> 150.txt 转换完成
158.json --> 158.txt 转换完成
162.json --> 162.txt 转换完成
17.json --> 17.txt 转换完成
174.json --> 174.txt 转换完成
179.json --> 179.txt 转换完成
184.json --> 184.txt 转换完成
189.json --> 189.txt 转换完成
190.json --> 190.txt 转换完成
191.json --> 191.txt 转换完成
192.json --> 192.txt 转换完成
196.json --> 196.txt 转换完成
201.json --> 201.txt 转换完成
207.json --> 207.txt 转换完成
215.json --> 215.txt 转换完成
218.json --> 218.txt 转换完成
222.json --> 222.txt 转换完成
226.json --> 226.txt 转换完成
231.json --> 231.txt 转换完成
24.json --> 24.txt 转换完成
241.json --> 241.txt 转换完成
245.json --> 245.txt 转换完成
246.json --> 246.txt 转换完成
258.json --> 258.txt 转换完成
26.json --> 26.txt 转换完成
269.json --> 269.txt 转换完成
271.json --> 271.txt 转换完成
274.json --> 274.txt 转换完成
275.json --> 275.txt 转换完成
280.json --> 280.txt 转

In [22]:
os.chdir('../../')

## 删除labelme格式的标注文件

In [23]:
!rm -rf labelme_jsons

## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [33]:
!find . -iname '__MACOSX'

In [34]:
!find . -iname '.DS_Store'

In [35]:
!find . -iname '.ipynb_checkpoints'

### 删除多余文件

In [36]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [37]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [38]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [39]:
!find . -iname '__MACOSX'

In [40]:
!find . -iname '.DS_Store'

In [41]:
!find . -iname '.ipynb_checkpoints'

## 得到完整的YOLO格式数据集